In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import time
import numpy as np
from functools import partial
import itertools


In [3]:
import pydrake
import meshcat

from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra, IrisInConfigurationSpace, IrisOptions
from pydrake.common import FindResourceOrThrow
from pydrake.geometry import SceneGraph
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.optimization import CalcGridPointsOptions, Toppra
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.multibody.tree import RevoluteJoint
from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
from pydrake.solvers.mosek import MosekSolver
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.primitives import TrajectorySource
from pydrake.trajectories import PiecewisePolynomial
from pydrake.all import Variable, Expression, RotationMatrix
from pydrake.all import MultibodyPositionToGeometryPose, ConnectMeshcatVisualizer, Role, Sphere
from pydrake.all import (
    ConvexSet, HPolyhedron, Hyperellipsoid,
    MathematicalProgram, Solve, le, IpoptSolver,
    Role, Sphere, VPolytope,
    Iris, IrisOptions, MakeIrisObstacles, Variable
)
from pydrake.all import (
    eq, SnoptSolver,
    Sphere, Ellipsoid, GeometrySet,
    RigidBody_, AutoDiffXd, initializeAutoDiff, InverseKinematics
)

import pydrake.symbolic as sym
import symbolic_parsing_helpers as symHelpers

from meshcat import Visualizer

import mcubes


# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])
proc2, zmq_url2, web_url2 = start_zmq_server_as_subprocess(server_args=[])

# Sporadically need to run `pkill -f meshcat`

from symbolic_parsing_helpers import generate_rationalize_trig_expr_rules
from symbolic_parsing_helpers import xreplace
from symbolic_parsing_helpers import NotRationalFunctionException
# from symbolic_parsing_helpers import rationalize_trig_expr
from pydrake.all import RationalFunction
snopt = SnoptSolver()
def xreplace(expr, rules):
        if isinstance(expr, float) or isinstance(expr, sym.Variable):
            return expr
        assert isinstance(expr, sym.Expression), expr
        for old, new in rules:
            if expr.EqualTo(old):
                return new
        ctor, old_args = expr.Deconstruct()
        new_args = [xreplace(e, rules) for e in old_args]
        return ctor(*new_args)

# Build simulation Environment

In [4]:
simple_collision = True
# gripper_welded = True

vis = Visualizer(zmq_url=zmq_url)
vis.delete()
vis2 = Visualizer(zmq_url=zmq_url2)
vis2.delete()

display(vis.jupyter_cell())

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

directives_file = FindResourceOrThrow("drake/sandbox/planar_iiwa_simple_collision_welded_gripper.yaml") \
    if simple_collision else FindResourceOrThrow("drake/sandbox/planar_iiwa_dense_collision_welded_gripper.yaml")
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)

q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(models[0].model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

plant.Finalize()

visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, delete_prefix_on_load=False)

diagram = builder.Build()
visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.Publish(context)


    
ik = InverseKinematics(plant, plant_context)
collision_constraint = ik.AddMinimumDistanceConstraint(0.001, 0.01)




You can open the visualizer by visiting the following URL:
http://127.0.0.1:7010/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7011/static/


Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6010...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7010/static/
Connected to meshcat-server.


In [5]:
def visualize_trajectory(traj):
    builder = DiagramBuilder()

    scene_graph = builder.AddSystem(SceneGraph())
    plant = MultibodyPlant(time_step=0.0)
    plant.RegisterAsSourceForSceneGraph(scene_graph)
    parser = Parser(plant)
    parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
                "drake/manipulation/models/wsg_50_description/package.xml")))

    directives_file = FindResourceOrThrow("drake/sandbox/planar_iiwa_simple_collision_welded_gripper.yaml") \
        if simple_collision else FindResourceOrThrow("drake/sandbox/planar_iiwa_dense_collision_welded_gripper.yaml")
    directives = LoadModelDirectives(directives_file)
    models = ProcessModelDirectives(directives, plant, parser)

    q0 = [-0.2, -1.2, 1.6]
    index = 0
    for joint_index in plant.GetJointIndices(models[0].model_instance):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[index])
            index += 1

    plant.Finalize()

    to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(plant))
    builder.Connect(to_pose.get_output_port(), scene_graph.get_source_pose_port(plant.get_source_id()))

    traj_system = builder.AddSystem(TrajectorySource(traj))
    builder.Connect(traj_system.get_output_port(), to_pose.get_input_port())

    meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url)

    vis_diagram = builder.Build()

    simulator = Simulator(vis_diagram)
    meshcat.start_recording()
    simulator.AdvanceTo(traj.end_time())
    meshcat.publish_recording()
    with open("/tmp/spp_shelves.html", "w") as f:
        f.write(meshcat.vis.static_html())


# Construct Rational Kinematics

In [6]:
from pydrake.all import RationalForwardKinematics
forward_kin = RationalForwardKinematics(plant)
q_star = np.zeros(forward_kin.t().shape[0])
t = forward_kin.t()
def convert_RationalForwardPoseList_to_TransformExpressionList(pose_list):
    ret = []
    for p in pose_list:
        ret.append(p.asRigidTransformExpr())
    return ret

def convert_t_to_q(t):
    q =np.arctan2(2*t/(1+t**2), (1-t**2)/(1+t**2))
    return q


def eval_cons(q0, q1, q2, c, tol):
        return 1-1*float(c.evaluator().CheckSatisfied([q0, q1, q2], tol))
    
col_func_handle = partial(eval_cons, c=collision_constraint, tol=0.01)


def eval_cons_rational(t0, t1, t2, c, tol):
        q = convert_t_to_q(np.array([t0, t1, t2])) 
        return 1-1*float(c.evaluator().CheckSatisfied(q, tol))

col_func_handle_rational = partial(eval_cons_rational, c=collision_constraint, tol=0.01)

In [7]:
q_low = plant.GetPositionLowerLimits()
q_high = plant.GetPositionUpperLimits()
t_low = forward_kin.ComputeTValue(q_low,q_star)
t_high = forward_kin.ComputeTValue(q_high,q_star)
print(t_low)
print(t_high)

[-1.73205081 -1.73205081 -1.73205081]
[1.73205081 1.73205081 1.73205081]


## Marching Cubes Obstacles

In [8]:
N = 50
vertices, triangles = mcubes.marching_cubes_func(tuple(t_low), tuple(t_high), N, N, N, col_func_handle_rational, 0.5)
vis2["collision_constraint"].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            meshcat.geometry.MeshLambertMaterial(color=0xff0000, wireframe=True))
# q = q0.copy()
# showres(q)

# Define Iris In T-Space

In [9]:
#  Now IRIS in configuration space, using dReal to solve for the growth volume
# through the nonconvex kinematics.

from pydrake.all import (
    eq, SnoptSolver,
    Sphere, Ellipsoid, GeometrySet,
    RigidBody_, AutoDiffXd, initializeAutoDiff,
)


def TangentPlane(self, point):
    a = 2 * self.A().T @ self.A() @ (point - self.center())
    a = a / np.linalg.norm(a)
    b = a.dot(point)
    return a, b

Hyperellipsoid.TangentPlane = TangentPlane


query = scene_graph.get_query_output_port().Eval(scene_graph.GetMyContextFromRoot(context))

sym_plant = plant.ToSymbolic()
sym_context = sym_plant.CreateDefaultContext()

# For SNOPT test.
autodiff_plant = plant.ToAutoDiffXd()
autodiff_context = autodiff_plant.CreateDefaultContext()
snopt = SnoptSolver()

def MakeFromHPolyhedronOrEllipseSceneGraph(query, geom, expressed_in=None):
    shape = query.inspector().GetShape(geom)
    if isinstance(shape, (Sphere, Ellipsoid)):
        return Hyperellipsoid(query, geom, expressed_in)
    return HPolyhedron(query, geom, expressed_in)
def MakeFromVPolytopeOrEllipseSceneGraph(query, geom, expressed_in=None):
    shape = query.inspector().GetShape(geom)
    if isinstance(shape, (Sphere, Ellipsoid)):
        return Hyperellipsoid(query, geom, expressed_in)
    return VPolytope(query, geom, expressed_in)


dReal_polytope_tol = 1

def CheckVolume(E, bodyA, bodyB, setA, setB, A, b, volume):
    prog = MathematicalProgram()
    q = prog.NewContinuousVariables(plant.num_positions(), "q")
    prog.AddBoundingBoxConstraint(lb, ub, q)
    prog.AddLinearConstraint(A, b-np.inf, b, q)
    p_AA =  prog.NewContinuousVariables(3, "p_AA")
    p_BB = prog.NewContinuousVariables(3, "p_BB")
    if volume < np.inf:
        prog.AddConstraint((q-E.center()).T @ E.A().T @ E.A() @ (q-E.center()) <= volume)
    setA.AddPointInSetConstraints(prog, p_AA)
    setB.AddPointInSetConstraints(prog, p_BB)

    sym_plant.SetPositions(sym_context, q)
    X_WA = sym_plant.EvalBodyPoseInWorld(sym_context, bodyA)
    X_WB = sym_plant.EvalBodyPoseInWorld(sym_context, bodyB)
    # Add +0 pending https://github.com/RobotLocomotion/drake/issues/15216
    print(type(X_WA))
    p_WA = X_WA.multiply(p_AA+0)
    p_WB = X_WB.multiply(p_BB+0)
    prog.AddConstraint(eq(p_WA, p_WB))
    prog.SetSolverOption(dReal.id(), "precision", .9*dReal_polytope_tol)
    result = dReal.Solve(prog)
    return result.is_success(), result.GetSolution(q)


def GrowthVolume(E, bodyA, bodyB, setA, setB, A, b, guess=None):
    prog = MathematicalProgram()
    q = prog.NewContinuousVariables(plant.num_positions(), "q")

    if guess is not None:
        prog.SetInitialGuess(q, guess)

    prog.AddLinearConstraint(A, b-np.inf, b, q)
    p_AA = prog.NewContinuousVariables(3, "p_AA")
    p_BB = prog.NewContinuousVariables(3, "p_BB")
    setA.AddPointInSetConstraints(prog, p_AA)
    setB.AddPointInSetConstraints(prog, p_BB)
    prog.AddQuadraticErrorCost(E.A().T @ E.A(), E.center(), q)

    # TODO: Remove these.  They're for debugging only.
    #set_meshcat_object(vis['setA'],setA)
    #set_meshcat_object(vis['setB'],setB)
    print(type(p_AA[0]))
    if isinstance(bodyA, RigidBody_[AutoDiffXd]):
        # TODO: Update this to use MBP<double> and Jacobians if I'm going to keep it.
        def kinematics_constraint(vars):
            p_AA, p_BB, q = np.split(vars,[3,6])
            autodiff_plant.SetPositions(autodiff_context, q)
            X_WA = autodiff_plant.EvalBodyPoseInWorld(autodiff_context, bodyA)
            X_WB = autodiff_plant.EvalBodyPoseInWorld(autodiff_context, bodyB)
            p_WA = X_WA.multiply(p_AA)
            p_WB = X_WB.multiply(p_BB)
            return p_WA - p_WB
        prog.AddConstraint(kinematics_constraint, lb=[0,0,0], ub=[0,0,0],
                           vars=np.concatenate((p_AA,p_BB,q)))
        result = snopt.Solve(prog)

    else:
        # TODO: Construct these symbolic expressions once per body outside this method.
        # But I would have to substitute in the new q each time.
        # Better is to construct the prog once for each pair, and just update the cost?
        sym_plant.SetPositions(sym_context, q)
        X_WA = sym_plant.EvalBodyPoseInWorld(sym_context, bodyA)
        X_WB = sym_plant.EvalBodyPoseInWorld(sym_context, bodyB)
        # Add +0 pending https://github.com/RobotLocomotion/drake/issues/15216
        p_WA = X_WA.multiply(p_AA+0)
        
        p_WB = X_WB.multiply(p_BB+0)
        prog.AddConstraint(eq(p_WA, p_WB))
        result = snopt.Solve(prog)
        #result = dReal.Solve(prog)

    return result.is_success(), result.get_optimal_cost(), result.GetSolution(q)

def iris_cspace(query, point, require_containment_points=[], termination_threshold=2e-2, iteration_limit=100):
#     vis = Visualizer(zmq_url=zmq_url)
#     set_meshcat_object(vis['sample'], point, color=0x99dd99)

    ellipsoid_epsilon = 1e-1
    dim = plant.num_positions()
    lb = plant.GetPositionLowerLimits()
    ub = plant.GetPositionUpperLimits()
#     assert dim == 3 # need to update the volume once this changes
    volume_of_unit_sphere = 4.0*np.pi/3.0
    E = Hyperellipsoid(np.eye(3)/ellipsoid_epsilon, point)
#     set_meshcat_object(vis['ellipse'], E)
    best_volume = ellipsoid_epsilon**dim * volume_of_unit_sphere

    inspector = query.inspector()
    pairs = inspector.GetCollisionCandidates()

    P = HPolyhedron.MakeBox(lb, ub)
    A = np.vstack((P.A(), np.zeros((10*len(pairs),3))))  # allow up to 10 faces per pair.
    b = np.concatenate((P.b(), np.zeros(10*len(pairs))))

    geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
    sets = {geom:MakeFromHPolyhedronOrEllipseSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}

    use_autodiff = False
    use_dReal = False
    ad_bodies = {geom:autodiff_plant.GetBodyFromFrameId(inspector.GetFrameId(geom)) for geom in geom_ids}
    sym_bodies = {geom:sym_plant.GetBodyFromFrameId(inspector.GetFrameId(geom)) for geom in geom_ids}
    if use_autodiff:
        bodies = ad_bodies
    else:
        bodies = sym_bodies
    print
    iteration = 0
    num_faces = 2*len(lb)
    while True:
        ## Find separating hyperplanes

        for geomA, geomB in pairs:
            print(f"geomA={inspector.GetName(geomA)}, geomB={inspector.GetName(geomB)}")
            # Run snopt at the beginning
            while True:
                success, growth, qstar = GrowthVolume(E,
                    bodies[geomA], bodies[geomB],
                    sets[geomA], sets[geomB], A[:num_faces,:], b[:num_faces] - dReal_polytope_tol, point)
                if success:
                    print(f"snopt_example={qstar}, growth = {growth}")
                    # Add a face to the polytope
                    A[num_faces,:], b[num_faces] = E.TangentPlane(qstar)
                    num_faces += 1
                else:
                    break

            if use_dReal:
                tries = 0
                while True:
                    reachable, counter_example = CheckVolume(
                        E, sym_bodies[geomA], sym_bodies[geomB], sets[geomA], sets[geomB],
                        A[:num_faces,:], b[:num_faces] - dReal_polytope_tol, np.inf)
                    if not reachable:
                        print("unreachable")
                        break
                    else:
                        z = E.A() @ (counter_example - E.center())
                        dreal_growth = z.dot(z)
                        print(f"counter_example = {counter_example}, growth = {dreal_growth}")
                        success, growth, qstar = GrowthVolume(E,
                            bodies[geomA], bodies[geomB],
                            sets[geomA], sets[geomB], A[:num_faces,:], b[:num_faces] - dReal_polytope_tol, counter_example)
                        if success:
                            print(f"snopt_example={qstar}, growth = {growth}")
                            # Add a face to the polytope
                            A[num_faces,:], b[num_faces] = E.TangentPlane(qstar)
                            num_faces += 1
                        if np.all(A[:num_faces,:] @ counter_example <= b[:num_faces] - dReal_polytope_tol):
                            # Then also add the counter-example
                            A[num_faces,:], b[num_faces] = E.TangentPlane(counter_example)
                            num_faces += 1
                    tries += 1

        if any([np.any(A[:num_faces,:] @ q > b[:num_faces]) for q in require_containment_points]):
            print("terminating because a required containment point would have not been contained")
            break

        P = HPolyhedron(A[:num_faces,:],b[:num_faces])
#         set_meshcat_object(vis[f'polytope'], P, wireframe=True)

        E = P.MaximumVolumeInscribedEllipsoid()
#         set_meshcat_object(vis[f'ellipse'], E)
        print(iteration)

        iteration += 1
        if iteration >= iteration_limit:
            break

        volume = volume_of_unit_sphere / np.linalg.det(E.A())
        if volume - best_volume <= termination_threshold:
            break
        best_volume = volume

    return P


In [10]:
def GrowthVolumeRational(E, X_WA, X_WB, setA, setB, A, b, guess=None):
    snopt = SnoptSolver()
    prog = MathematicalProgram()
    t = forward_kin.t()
    prog.AddDecisionVariables(t)
    
    if guess is not None:
        prog.SetInitialGuess(t, guess)

    prog.AddLinearConstraint(A, b-np.inf, b, t)
    p_AA = prog.NewContinuousVariables(3, "p_AA")
    p_BB = prog.NewContinuousVariables(3, "p_BB")
    setA.AddPointInSetConstraints(prog, p_AA)
    setB.AddPointInSetConstraints(prog, p_BB)
    prog.AddQuadraticErrorCost(E.A().T @ E.A(), E.center(), t)

    p_WA = X_WA.multiply(p_AA+0)

    p_WB = X_WB.multiply(p_BB+0)

    prog.AddConstraint(eq(p_WA, p_WB))
    result = snopt.Solve(prog)
    return result.is_success(), result.get_optimal_cost(), result.GetSolution(t)


In [11]:
def iris_rational_space(query, point, require_containment_points=[], termination_threshold=2e-2, iteration_limit=100):
    dReal_polytope_tol = 1e-2
    ellipsoid_epsilon = 1e-1
    dim = plant.num_positions()
    lb = plant.GetPositionLowerLimits()
    rational_lb = forward_kin.ComputeTValue(lb, q_star)
    ub = plant.GetPositionUpperLimits()
    rational_ub = forward_kin.ComputeTValue(ub, q_star)
    volume_of_unit_sphere = 4.0*np.pi/3.0
    E = Hyperellipsoid(np.eye(3)/ellipsoid_epsilon, point)
    best_volume = ellipsoid_epsilon**dim * volume_of_unit_sphere
    
    link_poses_by_body_index_rat_pose = forward_kin.CalcLinkPoses(q_star, 
                                                         plant.world_body().index())
    X_WA_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)
    X_WB_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)

    inspector = query.inspector()
    pairs = inspector.GetCollisionCandidates()

    P = HPolyhedron.MakeBox(rational_lb, rational_ub)
    A = np.vstack((P.A(), np.zeros((10*len(pairs),3))))  # allow up to 10 faces per pair.
    b = np.concatenate((P.b(), np.zeros(10*len(pairs))))

    geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
    sets = {geom:MakeFromHPolyhedronOrEllipseSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}
    body_indexes_by_geom_id = {geom:
                               plant.GetBodyFromFrameId(inspector.GetFrameId(geom)).index() for geom in geom_ids} 
    
    #Turn onto true to certify regions using SOS at each iteration.
    certify = False
    # refine polytopes if not certified collision free
    refine = False and certify
        
    iteration = 0
    num_faces = 2*len(lb)
    while True:
        ## Find separating hyperplanes

        for geomA, geomB in pairs:
            print(f"geomA={inspector.GetName(geomA)}, geomB={inspector.GetName(geomB)}")
            # Run snopt at the beginning
            while True:
                X_WA = X_WA_list[int(body_indexes_by_geom_id[geomA])]
                X_WB = X_WB_list[int(body_indexes_by_geom_id[geomB])]
                success, growth, qstar = GrowthVolumeRational(E,
                    X_WA, X_WB,
                    sets[geomA], sets[geomB], 
                    A[:num_faces,:], b[:num_faces] - dReal_polytope_tol, 
                    point)
                if success:
                    print(f"snopt_example={qstar}, growth = {growth}")
                    # Add a face to the polytope
                    A[num_faces,:], b[num_faces] = E.TangentPlane(qstar)
                    num_faces += 1
                else:
                    break

            if certify:
                pass
            

        if any([np.any(A[:num_faces,:] @ t > b[:num_faces]) for t in require_containment_points]):
            print("terminating because a required containment point would have not been contained")
            break

        P = HPolyhedron(A[:num_faces,:],b[:num_faces])

        E = P.MaximumVolumeInscribedEllipsoid()
        print(iteration)

        iteration += 1
        if iteration >= iteration_limit:
            break

        volume = volume_of_unit_sphere / np.linalg.det(E.A())
        if volume - best_volume <= termination_threshold:
            break
        best_volume = volume

    return P

In [12]:
seed_points = np.tan(np.array([[0.0, -2.016, 1.975], # in tight
                        [-1, -2, 0.5],        # neutral pose
                        [0.3, -0.8, 0.5],     # above shelf
                        [0.25, -1.6, -0.25],  # in shelf 1
                        [0.07, -1.8, -0.2],   # leaving shelf 1
                        [-0.1, -2, -0.3]])    # out of shelf 1
                        /2)
for i in range(seed_points.shape[0]):
    vis2['seedpoints']["seedpoint"+str(i)].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0xFFB900))
    vis2['seedpoints']["seedpoint"+str(i)].set_transform(
                meshcat.transformations.translation_matrix(seed_points[i,:]))

# traj = PiecewisePolynomial.FirstOrderHold(np.array([0, 1]), np.array([seed_points[4], seed_points[1]]).T)
# visualize_trajectory(traj)

In [13]:

query = scene_graph.get_query_output_port().Eval(scene_graph.GetMyContextFromRoot(context))
regions = []
ellipses = []
for i in range(seed_points.shape[0]):
    
    start_time = time.time()
    hpoly = iris_rational_space(query, seed_points[i,:], require_containment_points=[seed_points[i,:]], iteration_limit=100)
#     hpoly = iris_cspace(query, seed_points[i,:], require_containment_points=[seed_points[i,:]], iteration_limit=100)
    ellipse = hpoly.MaximumVolumeInscribedEllipsoid()
    print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
          ellipse.center(), flush=True)
    regions.append(hpoly)
    ellipses.append(ellipse)

print("SUCCESS")

geomA=iiwa::link7, geomB=shelves::top
snopt_example=[ 0.25686152 -0.44650254  1.37305101], growth = 138.27790339845825
geomA=iiwa::link7, geomB=wsg::right_collision
geomA=wsg::left_collision, geomB=wsg::right_collision
geomA=wsg::right_collision, geomB=shelves::shelf_upper
snopt_example=[ 0.14471298 -0.66482365  0.73909564], growth = 147.06126401711242
snopt_example=[ 0.26684134 -0.53520202  0.93115289], growth = 151.49619658550938
geomA=wsg::body_collision, geomB=shelves::top
geomA=wsg::body_collision, geomB=shelves::shelf_upper
snopt_example=[ 0.32551918 -0.53379534  1.10297422], growth = 138.142516784699
geomA=wsg::left_collision, geomB=shelves::top
geomA=wsg::right_collision, geomB=shelves::top
geomA=iiwa::link7, geomB=shelves::shelf_lower
snopt_example=[ 0.5119587  -0.80097613  1.4041356 ], growth = 88.94239682323666
geomA=wsg::body_collision, geomB=shelves::shelf_lower
geomA=wsg::left_collision, geomB=shelves::shelf_lower
snopt_example=[ 0.27896158 -0.8160305   0.60048261], growt

In [14]:
print(regions)
poly_to_cert = regions[2]
Atmp = poly_to_cert.A()
btmp = poly_to_cert.b()
poly_to_cert = HPolyhedron(Atmp, btmp-0.3)

[<pydrake.geometry.optimization.HPolyhedron object at 0x7f48fdd04db0>, <pydrake.geometry.optimization.HPolyhedron object at 0x7f4974316c30>, <pydrake.geometry.optimization.HPolyhedron object at 0x7f49acd28670>, <pydrake.geometry.optimization.HPolyhedron object at 0x7f48fdcb8c70>, <pydrake.geometry.optimization.HPolyhedron object at 0x7f48ff58ed30>, <pydrake.geometry.optimization.HPolyhedron object at 0x7f48fe593070>]


## Plot Regions

In [15]:
def inpolycheck(q0,q1,q2, A, b):
    q = np.array([q0, q1, q2])
    res = np.min(1.0*(A@q-b<=0))
    #print(res)
    return res

meshes = []
col1 = 0x002BFF
col2 = 0x3EFF00 
idx = 0
step = 1/(1.0*len(regions))
R = np.array([[1, 0, 0],[0, 0, 1],[0,1, 0]])

order = (0,1,2)
for i, region in enumerate(regions):
    A = region.A()[:,order]
    b = region.b()

    col_hand = partial(inpolycheck, A=A, b=b)
    vertices, triangles = mcubes.marching_cubes_func(tuple(t_low), tuple(t_high), 50, 50, 50, col_hand, 0.5)
    if i >= len(regions):
        regstr = "poly_to_cert"
        ellstr = "ell_to_cert"
    else:
        regstr = "regions" +str(idx)
        ellstr = "ellipse" +str(idx)
    mat = meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=True)
    mat.opacity = 0.3
#     vis2['regions'][regstr].set_object(
#             meshcat.geometry.TriangularMeshGeometry((R@vertices.T).T, triangles),
#             mat)
    vis2['regions'][regstr].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            mat)


    # C = ellipses[idx].A()
    # d = ellipses[idx].center()
    # radii, R = np.linalg.eig(C.T@C)
    # R[:,0] = R[:,0]*np.linalg.det(R)
    # Rot = RotationMatrix(R)
 
    # transf = RigidTransform(Rot, d)
    # mat = meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=True)
    # mat.opacity = 0.15
    # # vis2['ellipses'][ellstr].set_object(
    #         meshcat.geometry.Ellipsoid(np.divide(1,np.sqrt(radii))),
    #         meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=True))

    # vis2['ellipses'][ellstr].set_transform(transf.GetAsMatrix4())
    
    idx+=1
A_poly = poly_to_cert.A()         
b_poly = poly_to_cert.b()         
col_hand = partial(inpolycheck, A=A, b=b)
vertices, triangles = mcubes.marching_cubes_func(tuple(q_low), tuple(q_high), 50, 50, 50, col_hand, 0.5)
mat = meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=True)
mat.opacity = 0.3
vis2['poly_to_cert'].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            mat)
    

# Get all the bodies and rational poses

In [ ]:
link_poses_by_body_index_rat_pose = forward_kin.CalcLinkPoses(q_star, 
                                                         plant.world_body().index())
X_WA_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)
X_WB_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)

inspector = query.inspector()
pairs = inspector.GetCollisionCandidates()


geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
HPolyhedronSets = {geom:MakeFromHPolyhedronOrEllipseSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}
VPolyhedronSets = {geom:MakeFromVPolytopeOrEllipseSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}



body_indexes_by_geom_id = {geom:
                           plant.GetBodyFromFrameId(inspector.GetFrameId(geom)).index() for geom in geom_ids} 

geomA, geomB = next(iter(pairs))
X_WA = X_WA_list[int(body_indexes_by_geom_id[geomA])]
X_WB = X_WB_list[int(body_indexes_by_geom_id[geomB])]
for i, p in enumerate(pairs):
    geomA, geomB = p
    X_WA = X_WA_list[int(body_indexes_by_geom_id[geomA])]
    X_WB = X_WB_list[int(body_indexes_by_geom_id[geomB])]
    if i >= 15:
        break
# print(X_WA)
# print(X_WB)
bodyA, bodyB = VPolyhedronSets[geomA], VPolyhedronSets[geomB]

In [ ]:
vertex1 = X_WA@RigidTransform(RotationMatrix(), bodyA.vertices()[:,0]).translation()
vertex2 = X_WB@RigidTransform(RotationMatrix(),bodyB.vertices()[:,0]).translation()
base_point = X_WB@RigidTransform(RotationMatrix(), bodyB.vertices().mean(axis = 1)).translation()
# print(vertex1)
# print(vertex2)
print(base_point)
# print(bodyA.PointInSet(basePoint))

[<Expression "0.80000000000000004"> <Expression "0">
 <Expression "0.53115000000000001">]


In [ ]:
def convert_Expression_Vect_to_Rational(vect):
    ret = vect.copy()
    for i, v in enumerate(vect):
        ret[i] = symHelpers.RationalFunctionFromExpression(v)
    return np.array(ret)


rat_vertex1 = convert_Expression_Vect_to_Rational(vertex1)
rat_vertex2 = convert_Expression_Vect_to_Rational(vertex2)
rat_base_point = convert_Expression_Vect_to_Rational(base_point)
# print(rat_vert[0].denominator())

#randomly check if expressions are equal
tmp1 = rat_vertex1[0].ToExpression()
tmp2 = vertex1[0]
equal = True
for i in range(1000):
    test = 100*np.random.randn(len(tmp2.GetVariables()))
    ret1 = tmp1.Evaluate(dict(zip(tmp1.GetVariables(),test)))
    ret2 = tmp2.Evaluate(dict(zip(tmp2.GetVariables(),test)))
    equal = equal and np.isclose(ret1,ret2)
    if not equal:
        print(ret1)
        print(ret2)
        break
print(equal)
print(tmp1.Expand().EqualTo(tmp2.Expand()))

True
False


In [ ]:
def ExprVectIsAllConstant(vect):
    for v in vect:
        if v.get_kind() != sym.ExpressionKind.Constant:
            return False
    return True
def RatVectIsAllConstant(vect):
    for v in vect:
        tmp = v.ToExpression()
        if tmp.get_kind() != sym.ExpressionKind.Constant:
            return False
    return True
print(ExprVectIsAllConstant(vertex1))
print(RatVectIsAllConstant(rat_vertex1))
print(ExprVectIsAllConstant(vertex2))
print(RatVectIsAllConstant(rat_vertex2))

False
False
True
True


In [ ]:
def RationalVectHasCommonDenominatorAcrossVect(vect):
    ret = True
    for i in range(vect.shape[0]-1):
        tmp1 = vect[i].denominator()
        tmp2 = vect[i+1].denominator()
        ret = ret and tmp1.CoefficientsAlmostEqual(tmp2, 1e-10)
        ret = tmp1.EqualTo(tmp2)
    return ret

In [ ]:
print(tmp2.GetVariables())

{t[0], t[1], t[2]}


In [ ]:
prog = MathematicalProgram()
print(prog)
def addSeparatingVertexSeparatingHyperplaneConstraint(prog, vertex1, vertex2, base_point, plane_number = None):
    plane_name = "a" + str(plane_number) if plane_number is not None else "a"
    a = prog.NewContinuousVariables(vertex1.shape[0], plane_name)
    vect1 = vertex1 - base_point
    vect2 = vertex2 - base_point
    prog.AddLinearConstraint(a@vect1 <= 1)
    prog.AddLinearConstraint(a@vect2 >= 1)
    return prog
# prog = addSeparatingVertexSeparatingHyperplaneConstraint(prog, vertex1, vertex2, base_point, plane_number = 1)

a2 = prog.NewContinuousVariables(vertex1.shape[0], "a2")
convSolver = MosekSolver()
result = convSolver.Solve(prog)
print(result.is_success())
print(result.GetSolution())

No decision variables.

True
[0. 0. 0.]


In [ ]:
prog = MathematicalProgram()
convSolver = MosekSolver()
a2 = prog.NewContinuousVariables(vertex1.shape[0], "a2")
convSolver = MosekSolver()

def polynomialHyperplaneConstraintPolynomial(t, leq_or_geq, rat_vertex, rat_base_point, plane):
    vect = rat_vertex - rat_base_point
    if not RationalVectHasCommonDenominatorAcrossVect(vect):
        raise ValueError("vertex needs to have same denominator across entries")
    numerator_vect = np.array([v.numerator() for v in vect])
    prod = 0
    for i in range(numerator_vect.shape[0]):
        prod += numerator_vect[i]*plane[i]
    if leq_or_geq == 'leq':
        poly_sos_expr = -prod+vect[0].denominator()
    elif leq_or_geq == 'geq':
        poly_sos_expr = prod-vect[0].denominator()
    else:
        raise ValueError('leq_or_geq must be leq or geq not {}'.format(leq_or_geq))
    poly_sos_expr.SetIndeterminates(sym.Variables(t))
    return poly_sos_expr

def polynomialHyperplaneConstraint2(prog, plane, leq_or_geq, rat_vertex, rat_base_point, t,  
                                  t_Polyhedron_Constraint, lagrange_mult_degree = 4):
    method_start_time = time.time()
    start_time = time.time()
    poly_sos_expr = polynomialHyperplaneConstraintPolynomial(t, leq_or_geq, rat_vertex, rat_base_point, plane)
    print("Time to create poly_sos_expr: {}".format(time.time()-start_time))
    cone_rhs, _ = prog.NewSosPolynomial(sym.Variables(t), lagrange_mult_degree)
    A = t_Polyhedron_Constraint.A()
    b = t_Polyhedron_Constraint.b()
    for i in range(A.shape[0]):
        start_time = time.time()
        gi = sym.Polynomial(A[i,:]@t-b[i])
        lagrange_mult_poly, Gram_matrix = prog.NewSosPolynomial(sym.Variables(t), lagrange_mult_degree)
        cone_rhs += lagrange_mult_poly*gi
        print("Time to add constraint {}/{}: {}".format(i+1, A.shape[0], (time.time() - start_time)))
    start_time = time.time()
    lagrange_mult_poly, Gram_matrix = prog.NewSosPolynomial(sym.Variables(t), lagrange_mult_degree)
    cone_rhs += lagrange_mult_poly*poly_sos_expr
    cone_rhs.SetIndeterminates(sym.Variables(t))
    print("Time to add constraint poly_sos_expr: {}".format(i+1, (time.time() - start_time)))
    
    print(cone_rhs.indeterminates())
    print(prog.indeterminates())
    # print(cone_rhs)
    start_time  = time.time()
    prog.AddEqualityConstraintBetweenPolynomials(sym.Polynomial(-1), cone_rhs)
    print("Time to add eq constraint: {}".format(time.time() - start_time))
    print("Time to run method {}".format(time.time() - method_start_times))
    return prog, cone_rhs

def polynomialHyperplaneConstraint(prog, plane, leq_or_geq, rat_vertex, rat_base_point, t,  
                                  t_Polyhedron_Constraint, lagrange_mult_degree = 4, cone_level = 1):
    method_start_time = time.time()
    start_time = time.time()
    poly_sos_expr = polynomialHyperplaneConstraintPolynomial(t, leq_or_geq, rat_vertex, rat_base_point, plane)
    prog = add_Positivstellensatz_polytope_infeasibility_constraint(prog, t, -poly_sos_expr, t_Polyhedron_Constraint, lagrange_mult_degree, cone_level = -1)
    
    return prog
    
def constantHyperplaneConstraint(prog, plane, leq_or_geq, vertex, base_point):
    vect = vertex - base_point
    if leq_or_geq == 'leq':
        prog.AddLinearConstraint(plane@vect <= 1)
    elif leq_or_geq == 'geq':
        prog.AddLinearConstraint(plane@vect >= 1)
    else:
        raise ValueError('leq_or_geq must be leq or geq not {}'.format(leq_or_geq))
    return prog

def addHyperplaneConstraint(prog, plane, leq_or_geq, vertex, base_point, t, t_Polyhedron_Constraint, lagrange_mult_degree = 4):
    if ExprVectIsAllConstant(vertex) and ExprVectIsAllConstant(base_point):
        prog = constantHyperplaneConstraint(prog, plane, leq_or_geq, vertex, base_point)
    else:
        rat_vertex = convert_Expression_Vect_to_Rational(vertex)
        rat_base_point = convert_Expression_Vect_to_Rational(base_point)
        prog = polynomialHyperplaneConstraint(prog, plane, leq_or_geq, rat_vertex, rat_base_point, t,  
                                  t_Polyhedron_Constraint, lagrange_mult_degree)
                                
    return prog

def addSeparatingHyperplaneConstraint(prog, plane, vertex1, vertex2, base_point, t, t_Polyhedron_Constraint, lagrange_mult_degree = 4):
    # if plane_name is not None:
    #     plane = prog.NewContinuousVariables(vertex1.shape[0], plane_name)
    # else:
    #     plane = prog.NewContinuousVariables(vertex1.shape[0])
    prog = addHyperplaneConstraint(prog, plane, 'leq', vertex1, base_point, t, t_Polyhedron_Constraint, lagrange_mult_degree)
    prog = addHyperplaneConstraint(prog, plane, 'geq', vertex2, base_point, t, t_Polyhedron_Constraint, lagrange_mult_degree)
    return prog
    
def new_member_of_cone_generated_by_polytope(prog, t, polytope, max_deg, cone_level = -1):
    A = polytope.A()
    b = polytope.b()
    n = b.shape[0]
    poly, _ = prog.NewSosPolynomial(sym.Variables(t), max_deg)
    # print(type(A[0,:]@t - b))
    fi = [sym.Polynomial(A[i,:]@t - b[i]) for i in range(n)]
    if cone_level == -1 or cone_level > n:
        cone_level = n
    for lvl in range(1,cone_level+1):
        subsets = itertools.combinations(range(n),lvl)
        
        for s in subsets:
            term, _ = prog.NewSosPolynomial(sym.Variables(t), max_deg)
            cur_facts = [fi[i] for i in s]
            for f in cur_facts:
                term *= f 
            poly+= term
    poly.SetIndeterminates(sym.Variables(t))

    return prog, poly

def add_Positivstellensatz_polytope_infeasibility_constraint(prog, t, poly_to_cert, polytope, max_deg, cone_level = -1):
    prog, q1 = new_member_of_cone_generated_by_polytope(prog, t, polytope, max_deg, cone_level = cone_level)  
    prog, q2 = new_member_of_cone_generated_by_polytope(prog, t, polytope, max_deg, cone_level = cone_level)
    
    condition_poly = q2-q1*poly_to_cert
    condition_poly.SetIndeterminates(sym.Variables(t))

    prog.AddEqualityConstraintBetweenPolynomials(poly_to_cert, sym.Polynomial(-1))
    return prog
i = 4
tmp_polytope = HPolyhedron(poly_to_cert.A()[:i,:], poly_to_cert.b()[:i])

# cone_member = add_member_of_cone_generated_by_polytope(prog,t, tmp_polytope, 2, cone_level = -1)
prog.AddIndeterminates(t)
prog = add_Positivstellensatz_polytope_infeasibility_constraint(prog, t, sym.Polynomial(1), tmp_polytope, 2, cone_level = -1)
print("Prog start solving")
result = convSolver.Solve(prog)
print(result.is_success())


Prog start solving
False


In [ ]:
prog = MathematicalProgram()
prog.AddIndeterminates(t)
plane = prog.NewContinuousVariables(3, 'a')
prog= polynomialHyperplaneConstraint(prog, plane, 'leq', rat_vertex1, rat_base_point, t,  
                                  tmp_polytope, lagrange_mult_degree = 2, cone_level = 0)
# poly = polynomialHyperplaneConstraintPolynomial(t, 'leq', rat_vertex1, rat_base_point, a2)

# prog, cone_rhs = polynomialHyperplaneConstraint(prog, a2, 'leq', rat_vertex1, rat_base_point, t,  
#                                   poly_to_cert, lagrange_mult_degree = 6)

# prog = constantHyperplaneConstraint(prog, a2, 'geq', vertex2, base_point)

# prog = addHyperplaneConstraint(prog, a2, 'geq', vertex1, base_point, t, poly_to_cert, lagrange_mult_degree = 4)
# prog = addHyperplaneConstraint(prog, a2, 'leq', vertex2, base_point, t, poly_to_cert, lagrange_mult_degree = 4)

# prog, plane = addSeparatingHyperplaneConstraint(prog, plane, vertex2, vertex1, base_point, t, poly_to_cert, lagrange_mult_degree= 6, plane_name = "HELLO")

# print("Prog start solving")
# result = convSolver.Solve(prog)
# print(result.is_success())

# print(result.GetSolution(plane))
# details = result.get_solver_details()
# print(details.rescode)
# print(details.solution_status)

In [58]:
mono_map = (cone_rhs.monomial_to_coefficient_map())
print(len(mono_map.keys()))


1242


In [ ]:
def cert_safe_poly_two_body(t, poly_to_cert, X_WA, VSetA, X_WB, VSetB, lagrange_mult_degree= 6):
    prog = MathematicalProgram()
    plane = prog.NewContinuousVariables(VSetA.vertices()[:,0].shape[0])
    vertex1 = X_WA@RigidTransform(RotationMatrix(), VSetA.vertices()[:,0]).translation()
    if ExprVectIsAllConstant(vertex1):
        base_point = X_WA@RigidTransform(RotationMatrix(), VSetA.vertices().mean(axis = 1)).translation()
        order = (0,1)
    else:
        base_point = X_WB@RigidTransform(RotationMatrix(), VSetB.vertices().mean(axis = 1)).translation()
        order = (1,0)
    body_tup = VSetA, VSetB
    # safe = True
    for i in range(VSetA.vertices().shape[1]):
        vertexA =X_WA@RigidTransform(RotationMatrix(), VSetA.vertices()[:,i]).translation()
        for j in range(VSetB.vertices().shape[1]):
            vertexB = X_WB@RigidTransform(RotationMatrix(), VSetB.vertices()[:,j]).translation()
            vert_tup = vertexA, vertexB
            vertex1, vertex2 =  vert_tup[order[0]], vert_tup[order[1]]
            prog = addSeparatingHyperplaneConstraint(prog, plane, vertex1, vertex2, base_point, t, poly_to_cert, lagrange_mult_degree)
    print("Problem Formed")
    result = convSolver.Solve(prog)
    print("Problem Solved")
    safe = result.is_success()


    return safe

is_safe = cert_safe_poly_two_body(t, poly_to_cert, X_WA, bodyA, X_WB, bodyB, lagrange_mult_degree=2)
print(is_safe)
# print(bodyB.vertices())


RuntimeError: Expression (-1 - Symmetric(3,3) + 1.4320508075624863 * Symmetric(3,3) + 1.4320508075624863 * Symmetric(3,3) + 1.4320508075624863 * Symmetric(3,3) + 1.4320508075624863 * Symmetric(3,3) + 1.4320508075624863 * Symmetric(3,3) + 1.4320508075624863 * Symmetric(3,3) + 0.14176029762054076 * Symmetric(3,3) - 0.30267274614551393 * Symmetric(3,3) + 0.22799920099743681 * Symmetric(3,3) + 0.51860095396791261 * Symmetric(3,3) - (Symmetric(3,3) * (-1 - 0.78999999607819926 * x(0) - 0.054000006535850541 * x(1) + 0.96198299999999981 * x(2)))) is non-linear.

In [ ]:
prog = MathematicalProgram()
convSolver = MosekSolver()
X_WA_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)
X_WB_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)

inspector = query.inspector()
pairs = inspector.GetCollisionCandidates()


geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
HPolyhedronSets = {geom:MakeFromHPolyhedronOrEllipseSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}
VPolyhedronSets = {geom:MakeFromVPolytopeOrEllipseSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}



body_indexes_by_geom_id = {geom:
                           plant.GetBodyFromFrameId(inspector.GetFrameId(geom)).index() for geom in geom_ids} 
kept_regions = []
for i, poly_to_cert in enumerate(regions):
    safe = True
    for geomA, geomB in pairs:
        X_WA = X_WA_list[int(body_indexes_by_geom_id[geomA])]
        X_WB = X_WB_list[int(body_indexes_by_geom_id[geomB])]
        if ExprVectIsAllConstant(X_WA.translation()) or ExprVectIsAllConstant(X_WB.translation()):
            bodyA, bodyB = VPolyhedronSets[geomA], VPolyhedronSets[geomB]
            safe = safe and  cert_safe_poly_two_body(t, poly_to_cert, X_WA, bodyA, X_WB, bodyB, lagrange_mult_degree=10)
        if not safe:
            continue
    if safe:
        kept_regions += poly_to_cert
    print("Completed {}/{} regions".format(i+1,len(regions)))
    print("Kept {}/{}".format(len(kept_regions), i+1))
    print()

Completed 1/6 regions
Kept 0/1

Completed 2/6 regions
Kept 0/2



KeyboardInterrupt: 

In [ ]:
print(X_WA.translation()[0])

((2.9359403253526209e-13 + 2.184266 * t[0] - 1.3442660000000002 * t[1] + 0.54426600000000003 * t[2] + 1.1745110862183144e-12 * (t[0] * t[1]) + 2.1770640000000001 * (t[0] * t[1] * t[2]) - 1.1749617562404005e-12 * (t[0] * t[1] * pow(t[2], 2)) - 0.50426600000000021 * (t[0] * pow(t[1], 2)) + 1.1747364212293574e-12 * (t[0] * pow(t[1], 2) * t[2]) + 0.58426599999999984 * (t[0] * pow(t[1], 2) * pow(t[2], 2)) - 1.1747364212293574e-12 * (t[0] * t[2]) + 1.0957339999999998 * (t[0] * pow(t[2], 2)) + 1.3442660000000002 * (pow(t[0], 2) * t[1]) - 1.1747364212293574e-12 * (pow(t[0], 2) * t[1] * t[2]) + 0.25573400000000002 * (pow(t[0], 2) * t[1] * pow(t[2], 2)) + 2.9364681481230529e-13 * (pow(t[0], 2) * pow(t[1], 2)) + 0.54426600000000003 * (pow(t[0], 2) * pow(t[1], 2) * t[2]) - 2.9372139580237342e-13 * (pow(t[0], 2) * pow(t[1], 2) * pow(t[2], 2)) - 0.54426600000000003 * (pow(t[0], 2) * t[2]) + 2.9375948231782687e-13 * (pow(t[0], 2) * pow(t[2], 2)) + 1.1747364212293574e-12 * (t[1] * t[2]) - 0.2557340000

## Do Path Planning

In [ ]:
def set_joint_angles(vals):
    plant.SetPositions(plant_context,vals)


In [ ]:
solve_path_planning = False
if solve_path_planning:
    start = seed_points[0,:]#convert_q_to_t(np.array([0.2, -1.6, 1.5]))
    target = seed_points[3,:]#convert_q_to_t(np.array([0.8,-0.9,1.5]))
    start_time = time.time()
    spp = BsplineTrajectoryThroughUnionOfHPolyhedra(start, target, regions)
    spp.set_max_velocity([.8, .8, .8])
    spp.set_extra_control_points_per_region(3)

    # print(spp.num_regions())
    traj = spp.Solve()
    print(time.time() - start_time)
    print(traj.start_time())
    print(traj.end_time())

9.287066221237183
0.0
2.906042767645163


In [ ]:
# Solve path planning
if solve_path_planning:

    #visualize
    maxit = 60
    pts = []
    for it in range(maxit):
        # print(convert_t_to_q(traj.value(it*traj.end_time()/maxit)))
        pts.append(traj.value(it*traj.end_time()/maxit))
        # pts.append(traj.value(it*traj.end_time()/maxit))
        mat = meshcat.geometry.MeshLambertMaterial(color=0xFFF812)
        mat.reflectivity = 1.0
        vis2['traj']['points' + str(it)].set_object(
                    meshcat.geometry.Sphere(0.06), mat)
        vis2['traj']['points' + str(it)].set_transform(
                    meshcat.transformations.translation_matrix(pts[-1].reshape(-1,)))
        
        set_joint_angles(pts[-1].reshape(-1,))
        tf_l2 = plant.EvalBodyPoseInWorld(plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(3)))
        R_l2 = tf_l2.rotation()
        tl_l2 = R_l2@np.array([0,0,0.9]) + tf_l2.translation()
        
        tf_la = plant.EvalBodyPoseInWorld(plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(4)))
        R_la = tf_la.rotation()
        tl_la = R_la@np.array([0,0,1.2]) + tf_la.translation()
        
        
        mat = meshcat.geometry.MeshLambertMaterial(color=0x0029F1)
        mat.reflectivity = 1.0
        vis['traj']['link2']['points' + str(it)].set_object(
                    meshcat.geometry.Sphere(0.02), mat)
        vis['traj']['link2']['points' + str(it)].set_transform(
                    meshcat.transformations.translation_matrix(tl_l2))
        mat = meshcat.geometry.MeshLambertMaterial(color=0x07F100)
        mat.reflectivity = 1.0
        vis['traj']['linka']['points' + str(it)].set_object(
                    meshcat.geometry.Sphere(0.02), mat)
        vis['traj']['linka']['points' + str(it)].set_transform(
                    meshcat.transformations.translation_matrix(tl_la))
    

In [ ]:
animate = False

if animate:
    idx = 0
    going_fwd = True
    steps = 1000
    time_points = np.linspace(0, traj.end_time(), steps) 

    def showres(q):
        set_joint_angles(q)
        # col = col_func_handle(*q)
        # t = convert_q_to_t(q)
        # if col:
        #     vis2["q"].set_object(
        #             meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0xFFB900))
        #     vis2["q"].set_transform(
        #             meshcat.transformations.translation_matrix(t))
        # else:
        #     vis2["q"].set_object(
        #             meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0x3EFF00))
        #     vis2["q"].set_transform(
        #             meshcat.transformations.translation_matrix(t))
        diagram.Publish(context)
        # print("              ", end = "\r")
        # print(t , end = "\r")

    while True:
        q = convert_t_to_q(traj.value(time_points[idx]))
        showres(q.reshape(-1,))
        if going_fwd:
            if idx + 1 < steps:
                idx += 1
            else:
                going_fwd = False
                idx -=1
        else:
            if idx-1 >= 0:
                idx -=1
            else:
                going_fwd = True
                idx +=1
            

ZMQError: Operation cannot be accomplished in current state